Just a test

In [15]:
import sys
import os
import json
import numpy as np
import random
import h5py

###############################################
def get_script_directory(n_levels_up=0, explicit_tmp_dir=None):
    main_dir = ''
    script_dir = ''
    include_dir = ''
    temp_dir = '/tmp'
    try:
        # Check if running in a standard Python script
        script_dir = os.path.dirname(os.path.abspath(__file__))
        # return script_dir
    except NameError:
        # If __file__ is not defined, we are likely in a Jupyter Notebook
        from IPython import get_ipython
        if 'IPKernelApp' in get_ipython().config:
            script_dir = os.getcwd()
        else:
            raise RuntimeError("Unable to determine the script directory")
    # cd up n_levels_up
    main_dir = script_dir
    for i in range(n_levels_up):
        main_dir = os.path.dirname(main_dir)
    include_dir = os.path.join(main_dir, 'include')
    # leave the temporary directory to the user
    if explicit_tmp_dir is not None:
        temp_dir = explicit_tmp_dir
        return main_dir, script_dir, include_dir, temp_dir
    # else we try the simplest way to get the temporary directory
    # determine if we run Linux, MacOS or Windows
    if sys.platform == 'linux':
        temp_dir = '/tmp'
    elif sys.platform == 'darwin':
        temp_dir = '/tmp'
    elif sys.platform == 'win32':
        temp_dir = os.environ['TEMP']
    else:
        raise RuntimeError("Unknown platform")

    # return all directories
    return main_dir, script_dir, include_dir, temp_dir
####################################
main_dir, script_dir, include_dir, temp_dir = get_script_directory(1)
data_dir = os.path.join(main_dir, 'data')
# add the include directory to the path
sys.path.append(include_dir)
import atss_file as atss
import atss_extended as atss_ext

## Here we go

In [14]:
# read all JSON files in the run directory
survey = 'Northern Mining'
site = 'Sarıçam'
run = '006'
run_dir = os.path.join(data_dir, 'Sarıçam', 'run_006')
channels = []
ts = []
files = []
for file in os.listdir(run_dir):
    if file.endswith('.json'):
        with open(os.path.join(run_dir, file)) as f:
            # get the file name without the extension from f
            files.append(os.path.splitext(f.name)[0])
            channels.append(atss.read_header(f.name))
            # in the real world you may read chunks of data
            ts.append(atss.read_data(f.name))

print("reading ", len(channels) , " files with ", atss.samples(files[0]), "samples each" )
print("array shape: ", ts[0].shape)



reading  5  files with  7912 samples each
array shape:  (7912,)


In [13]:
# delete the file if it exists
if os.path.exists('./demo_surveys.h5'):
    os.remove('./demo_surveys.h5')
with h5py.File('./demo_surveys.h5', 'w') as hdf:
        su = hdf.create_group(survey)
        cal = hdf.create_group(survey + '/calibration')
        s1 = hdf.create_group(su.name + '/' + site)
        # take a run - so we can later add data from other sensors / transmitters here
        r1 = hdf.create_group(s1.name + '/' + run)
        # loop over the channels and ts
        for i in range(len(channels)):
            header = channels[i]
            #data = ts[i]
            # create a group for each channel
            ch = r1.create_group(header['channel_type'])
            # create the datasets for the data ts[i]
            ds = ch.create_dataset('data', data = ts[i])
            # ds = ch.create_dataset('data', data = ts[i])
            # add the attributes to the dataset which are key-value pairs from the header
            #ds.attrs['channel_type'] = header['channel_type']
            #ds.attrs['serial'] = header['serial']
            # add the attributes to the group which are key-value pairs from the header
            
            for key in header.keys():
                # skip / ignore all "sensor_calibration" key
                if key == "sensor_calibration":
                     continue
                else:
                    try:
                        ch.attrs[key] = header[key]
                    except:
                        print('error adding attribute', key, header[key])   

        # close the file
        hdf.close()    
        
     
        


try to read

In [16]:
view_channel = 'Hx'
with h5py.File('./demo_surveys.h5', 'r') as hdf:
   all_h5_objs = []
   hdf.visit(all_h5_objs.append)
   all_groups   = [ obj for obj in all_h5_objs if isinstance(hdf[obj],h5py.Group) ]
   all_datasets = [ obj for obj in all_h5_objs if isinstance(hdf[obj],h5py.Dataset) ]
   # print(all_groups)
   # print(all_datasets)  # need some more time for fishing
   str = survey + '/' + site + '/' + run + '/' + view_channel
   get = hdf.get(str)
   # print the attributes of the dataset as key-value pairs
   for key in get.attrs.keys():
       print(key, get.attrs[key])
   # get the data from the dataset data
   hx = get['data']
   print("getting: ", len(hx), " samples")

  

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3874403866.py, line 9)